In [1]:
import fitz
import re

def extract_target_number(pdf_path):
    doc = fitz.open(pdf_path)

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        blocks = page.get_text("dict")["blocks"]

        for b in blocks:
            if "lines" in b:
                for line in b["lines"]:
                    for span in line["spans"]:
                        if (
                            'number' in b and b['number'] == 5
                            and 'size' in span and span['size'] == 12.0
                            and 'font' in span and span['font'] == 'Helvetica-Bold'
                            and page_num == 0  # Only on the first page
                        ):
                            match = re.search(r'\((\d+)\)', span["text"])
                            if match:
                                return int(match.group(1))

    doc.close()


In [2]:
import fitz
import re

def extract_text_by_font_pattern_stop(pdf_path):
    doc = fitz.open(pdf_path)
    extracted_text_list = []

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        blocks = page.get_text("dict")["blocks"]

        for b in blocks:
            if "lines" in b:
                for line in b["lines"]:
                    for span in line["spans"]:
                        if 'font' in span and span['font'] == 'Helvetica-Oblique':
                            # match = re.search(r'([^,]+)(?:,|$)', span["text"])
                            # match = re.search(r'^(.*?),', span["text"])
                            # match = re.search(r'^(.*?)(?:,|$)', span["text"])
                            # result_string = re.sub(r', \d+ U\.S\. \d+$', '', span["text"])
                            result_string = re.sub(r', \d+ [A-Za-z. ]+ \d+$', '', span["text"])
                            # if match:
                            #     extracted_text_list.append(match.group(1).strip())
                            extracted_text_list.append(result_string.strip())
        if len(extracted_text_list) >= extract_target_number(pdf_path):
            break

    doc.close()
    return extracted_text_list

pdf_file_path = "D:/antitrust/verdict/Federal District (30).PDF"

result_list = extract_text_by_font_pattern_stop(pdf_file_path)

for i, text in enumerate(result_list):
    print(f'Text {i + 1}: {text}')

# 38 is gone

Text 1: Wm. Filene's Sons Co. v. Fashion Originators' Guild, Inc.
Text 2: United States v. Lake Asphalt & Petroleum Co.
Text 3: United States v. Allied Chem. Corp.
Text 4: United States v. Bituminous Concrete Assn., Inc.
Text 5: Union Leader Corp. v. Newspapers of New England, Inc.
Text 6: United States v. Grinnell Corp.
Text 7: Stanton v. Texaco, Inc.
Text 8: Liquilux Gas Services, Inc. v. Tropical Gas Co.
Text 9: Truxes v. Rolan Electric Corp.
Text 10: Boston Professional Hockey Asso. v. Cheevers
Text 11: Lang's Bowlarama, Inc. v. AMF, Inc.
Text 12: Anderson Foreign Motors, Inc. v. New England Toyota Distributors, Inc.
Text 13: Bass v. Boston Five Cent Sav. Bank
Text 14: Hew Corp. v. Tandy Corp.
Text 15: A. D. M. Corp. v. Sigma Instruments, Inc.
Text 16: Transitron Electronic Corp. v. Hughes Aircraft Co.
Text 17: Allied Int'l v. International Longshoremen's Ass'n
Text 18: Anderson Foreign Motors, Inc. v. New England Toyota Distributor, Inc.
Text 19: Grendel's Den v. Goodwin
Text 20: 

In [34]:


def get_index(pdf_path ,case_list):
    doc = fitz.open(pdf_path)
    i = 0
    a_list = []
    page_list = []
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        blocks = page.get_text("dict")["blocks"]

        for b in blocks:
            for i in range(len(case_list)):
                if 'number' in b and b['number'] == 1:
                    if "lines" in b:
                        for line in b["lines"]:
                            for span in line["spans"]:
                                if span["text"] == case_list[i]:
                                    a_list.append(i,{span["text"]})
                                    i+=1
    doc.close()
    return ({"title":a_list, "page": page_num})



In [35]:
# pdf_file_path = "D:/antitrust/verdict/Files (100).PDF"
pdf_file_path = "D:/antitrust/verdict/Federal District (30).PDF"
result_list = extract_text_by_font_pattern_stop(pdf_file_path)

In [37]:
result_list

["Wm. Filene's Sons Co. v. Fashion Originators' Guild, Inc.",
 'United States v. Lake Asphalt & Petroleum Co.',
 'United States v. Allied Chem. Corp.',
 'United States v. Bituminous Concrete Assn., Inc.',
 'Union Leader Corp. v. Newspapers of New England, Inc.',
 'United States v. Grinnell Corp.',
 'Stanton v. Texaco, Inc.',
 'Liquilux Gas Services, Inc. v. Tropical Gas Co.',
 'Truxes v. Rolan Electric Corp.',
 'Boston Professional Hockey Asso. v. Cheevers',
 "Lang's Bowlarama, Inc. v. AMF, Inc.",
 'Anderson Foreign Motors, Inc. v. New England Toyota Distributors, Inc.',
 'Bass v. Boston Five Cent Sav. Bank',
 'Hew Corp. v. Tandy Corp.',
 'A. D. M. Corp. v. Sigma Instruments, Inc.',
 'Transitron Electronic Corp. v. Hughes Aircraft Co.',
 "Allied Int'l v. International Longshoremen's Ass'n",
 'Anderson Foreign Motors, Inc. v. New England Toyota Distributor, Inc.',
 "Grendel's Den v. Goodwin",
 'Auburn News Co. v. Providence Journal Co.',
 'Baldini v. New England Greyhound Asso.',
 'Barr

In [33]:
for i, result in enumerate(result_texts, start=1):
    print(f'Index {i}: Title: {result["Title"]},\n         Court: {result["Court"]}')

Index 1: Title: Wm. Filene's Sons Co. v. Fashion Originators' Guild, Inc.,
         Court: District Court, D. Massachusetts
Index 2: Title: United States v. Lake Asphalt & Petroleum Co.,
         Court: United States District Court for the District of Massachusetts.
Index 3: Title: United States v. Allied Chem. Corp.,
         Court: United States District Court for the District of Massachusetts.
Index 4: Title: United States v. Bituminous Concrete Assn., Inc.,
         Court: United States District Court for the District of Massachusetts.
Index 5: Title: Union Leader Corp. v. Newspapers of New England, Inc.,
         Court: United States District Court for the District of Massachusetts
Index 6: Title: United States v. Grinnell Corp.,
         Court: United States District Court for the District of Rhode Island
Index 7: Title: Stanton v. Texaco, Inc.,
         Court: United States District Court for the District of Rhode Island
Index 8: Title: Liquilux Gas Services, Inc. v. Tropical Ga

In [39]:
import fitz

def get_texts_matching_cases(pdf_path, case_list):
    doc = fitz.open(pdf_path)
    matching_texts = []
    a_text = None  # Variable to store 'number': 1 text
    b_text = None  # Variable to store 'number': 2 text
    c_text = None  # Variable to store 'number': 3 text
    d_text = None  # Variable to store 'number': 4 text
    consider_number_2 = False  # Flag to indicate whether to consider 'number': 2 text
    
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        blocks = page.get_text("dict")["blocks"]

        for b in blocks:
            if 'number' in b and b['number'] == 1:
                if "lines" in b:
                    # Reset text variables for each new 'number': 1
                    a_text = None
                    b_text = None
                    c_text = None
                    d_text = None

                    for line in b["lines"]:
                        for span in line["spans"]:
                            if 'font' in span and span['font'] == 'Helvetica-BoldOblique':
                                for case_name in case_list:
                                    if span["text"].strip() == case_name.strip():
                                        a_text = span["text"].strip()
                                        # Set the flag to consider 'number': 2 and 'number': 3 text
                                        consider_number_2 = True
                                        break  # Stop looking for 'number': 1 text once found
            if consider_number_2 and 'number' in b and b['number'] == 2:
                if "lines" in b:
                    for line in b["lines"]:
                        for span in line["spans"]:
                            if ('font' in span and span['font'] == 'Helvetica') and ('size' in span and span['size'] == 10):
                                b_text = span["text"].strip()
            if consider_number_2 and 'number' in b and b['number'] == 3:
                if "lines" in b:
                    for line in b["lines"]:
                        for span in line["spans"]:
                            if ('font' in span and span['font'] == 'Helvetica') and ('size' in span and span['size'] == 10):
                                c_text = span["text"].strip()
            if consider_number_2 and 'number' in b and b['number'] == 4:
                if "lines" in b:
                    for line in b["lines"]:
                        for span in line["spans"]:
                            if ('font' in span and span['font'] == 'Helvetica') and ('size' in span and span['size'] == 10):
                                d_text = span["text"].strip()
                                # Stop considering 'number': 2, 'number': 3, and 'number': 4 text
                                consider_number_2 = False
                                break  # Stop looking for 'number': 4 text once found
        # Check if 'number': 1, 'number': 2, 'number': 3, and 'number': 4 text are present, then append to the result
        if a_text and b_text and c_text:
            matching_texts.append({'Title': a_text, 'Court': b_text, 'Date': c_text, 'No': d_text, 'page':page_num })
    
    doc.close()
    return matching_texts

# Example usage
result_texts = get_texts_matching_cases(pdf_file_path, result_list)

# Print the result
for i, result in enumerate(result_texts):
    print(f'Index {i + 1}: Title: {result["Title"]}, Court: {result["Court"]}, Date: {result["Date"]}, No: {result["No"]}, page: {result["page"]}')


Index 1: Title: Wm. Filene's Sons Co. v. Fashion Originators' Guild, Inc., Court: District Court, D. Massachusetts, Date: March 24, 1936, No: No. 4293, page: 7
Index 2: Title: United States v. Lake Asphalt & Petroleum Co., Court: United States District Court for the District of Massachusetts., Date: October 17, 1960., No: Civil Action No. 59-786-W., page: 16
Index 3: Title: United States v. Allied Chem. Corp., Court: United States District Court for the District of Massachusetts., Date: November 28, 1960., No: Civil Action No. 59-784-S., page: 20
Index 4: Title: United States v. Bituminous Concrete Assn., Inc., Court: United States District Court for the District of Massachusetts., Date: December 7, 1960., No: Civil Action No. 59-785-M., page: 23
Index 5: Title: Union Leader Corp. v. Newspapers of New England, Inc., Court: United States District Court for the District of Massachusetts, Date: June 10, 1963, No: Civ. A. No. 59-23, page: 27
Index 6: Title: United States v. Grinnell Corp.,